In [ ]:
import backtesting as bt
import pandas as pd
import numpy as np
import seaborn as sns
import yfinance as yf
import pandas_ta as ta
import datetime as dt

In [ ]:
data = yf.download('^NSEBANK',period='1d',interval='1m')

[*********************100%%**********************]  1 of 1 completed


In [ ]:
bb = ta.bbands(data.Close,20,2)
bb[bb['BBB_20_2.0']==0.35]

,BBL_20_2.0,BBM_20_2.0,BBU_20_2.0,BBB_20_2.0,BBP_20_2.0


In [ ]:
def BBands_indicator(data,length,sd):
  bbands= ta.bbands(data.Close.s,length,sd)
  return bbands.to_numpy().T[:3]

def BBands_width(data,length,sd):
  bbands = ta.bbands(data.Close.s,length,sd)
  return bbands.to_numpy().T[3]

def BBands_per(data,length,sd):
  bbands = ta.bbands(data.Close.s,length,sd)
  return bbands.to_numpy().T[4]

In [ ]:
class BBStrategy(bt.Strategy):

  length = 20
  std = 20
  sd = std/10
  stoploss = 0.9
  sp = stoploss/(100*10)
  bbb_limit = 10
  bbb_l = bbb_limit/100


  def init(self):
    self.bbands = self.I(BBands_indicator,self.data,self.length,self.sd)
    self.bbb = self.I(BBands_width,self.data,self.length,self.sd)
    self.bbp = self.I(BBands_per,self.data,self.length,self.sd)


  def next(self):
    price = self.data.Close[-1]
    high = self.data.High[-1]
    low = self.data.Low[-1]
    tick_time = self.data.index[-1]
    upper_band = self.bbands[2]
    middle_band = self.bbands[1]
    lower_band = self.bbands[0]
    #print(tick_time,price,upper_band[-1],middle_band[-1],lower_band[-1])
    bbb_ = self.bbb[-1] > self.bbb_l
    #print(self.bbb[-1],self.bbb_l,bbb_)

    if tick_time.time() > dt.time(hour=10,minute = 5) and bbb_:

      if low < lower_band[-1]:
        #print(tick_time,price,lower_band[-1])

        if not self.position.is_long:
          self.position.close()
          self.buy(size=15,sl=(1-self.sp)*price)
      if high > upper_band[-1]:
        #print(tick_time,price,upper_band[-1])

        if not self.position.is_short:
          self.position.close()
          self.sell(size=15,sl=(1+self.sp)*price)

    if tick_time.time() > dt.time(hour=14,minute=5):
      self.position.close()


In [ ]:
eng = bt.Backtest(data,BBStrategy,cash=4000000,commission=0.00024)
stats = eng.run()
stats

Start                     2024-04-16 09:15...
End                       2024-04-16 15:29...
Duration                      0 days 06:14:00
Exposure Time [%]                   72.266667
Equity Final [$]               3998047.018454
Equity Peak [$]                4002834.241963
Return [%]                          -0.048825
Buy & Hold Return [%]                0.510597
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                   -0.119596
Avg. Drawdown [%]                   -0.016579
Max. Drawdown Duration        0 days 02:09:00
Avg. Drawdown Duration        0 days 00:23:00
# Trades                                   22
Win Rate [%]                        36.363636
Best Trade [%]                       0.147887
Worst Trade [%]                     -0.122976
Avg. Trade [%]                    

In [ ]:
eng.plot()

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p1333', ...)

In [ ]:
opt_stats,heatmap = eng.optimize(
    length = range(3,30,2),
    std = range(1,30),
    bbb_limit = range(4,35),
    maximize = 'Win Rate [%]',
    return_heatmap= True
)

/usr/local/lib/python3.10/dist-packages/backtesting/backtesting.py:1488: UserWarning: Searching for best of 12586 configurations.
  output = _optimize_grid()


Backtest.optimize:   0%|          | 0/42 [00:00<?, ?it/s]

In [ ]:
opt_stats

AttributeError: '_Stats' object has no attribute 'bbb_l'

In [ ]:
opt_stats

Start                     2024-04-16 09:15...
End                       2024-04-16 15:29...
Duration                      0 days 06:14:00
Exposure Time [%]                        43.2
Equity Final [$]               4000553.050502
Equity Peak [$]                4000809.385937
Return [%]                           0.013826
Buy & Hold Return [%]                0.510597
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                   -0.034943
Avg. Drawdown [%]                   -0.017516
Max. Drawdown Duration        0 days 03:27:00
Avg. Drawdown Duration        0 days 01:21:00
# Trades                                    5
Win Rate [%]                             40.0
Best Trade [%]                        0.13197
Worst Trade [%]                     -0.037693
Avg. Trade [%]                    